<a href="https://colab.research.google.com/github/afroman32/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module1/LS_DS_121_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 2 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, ttest_1samp

In [67]:
# download the csv file
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'

--2019-12-09 22:42:31--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data.1’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.06s   

2019-12-09 22:42:32 (277 KB/s) - ‘house-votes-84.data.1’ saved [18171/18171]



In [0]:
# read in data to a dataframe and assign column titles
name_col = ['party', 'handicapped-infants', 'water-project', 'budget',
            'physician-fee-freeze','el-salvadaor-aid', 'religious-groups',
            'anti-satellite-ban', 'aid-to-contras', 'zx-missile', 'immigration',
            'synfuels', 'education', 'right-to-sue', 'crime', 'duty-free',
            'south-africa']
df = pd.read_csv('house-votes-84.data', header = None, names = name_col)

In [69]:
print(df.shape)
df.head()

(435, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvadaor-aid,religious-groups,anti-satellite-ban,aid-to-contras,zx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [70]:
# change data values so they can be used for statistical analysis
df = df.replace({'?':np.NaN, 'n':0, 'y':1})
df.head()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvadaor-aid,religious-groups,anti-satellite-ban,aid-to-contras,zx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [71]:
# check for null values
df.isnull().sum()

party                     0
handicapped-infants      12
water-project            48
budget                   11
physician-fee-freeze     11
el-salvadaor-aid         15
religious-groups         11
anti-satellite-ban       14
aid-to-contras           15
zx-missile               22
immigration               7
synfuels                 21
education                31
right-to-sue             25
crime                    17
duty-free                28
south-africa            104
dtype: int64

In [118]:
# create a subset with just republican part
republican = df[df.party == 'republican']
print(republican.shape)
republican.head()

(168, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvadaor-aid,religious-groups,anti-satellite-ban,aid-to-contras,zx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [119]:
# remove the party column as it is no longer needed to distinguish between parties
republican.drop(columns = 'party', axis = 1, inplace = True)
republican.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvadaor-aid,religious-groups,anti-satellite-ban,aid-to-contras,zx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [73]:
# create a subset for just democratic party
democrat = df[df.party == 'democrat']
print(democrat.shape)
democrat.head()

(267, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvadaor-aid,religious-groups,anti-satellite-ban,aid-to-contras,zx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [99]:
# remove the party column as it is no longer needed to distinguish between parties
democrat.drop(columns = 'party', axis = 1, inplace = True)
democrat.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvadaor-aid,religious-groups,anti-satellite-ban,aid-to-contras,zx-missile,immigration,synfuels,education,right-to-sue,crime,duty-free,south-africa
2,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [74]:
#  2 - sample compares the means of two samples, null is that they're the same
ttest_ind(republican['handicapped-infants'], democrat['handicapped-infants'], 
          nan_policy = 'omit')

Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18)

In [75]:
ttest_ind(republican['budget'], democrat['budget'], 
          nan_policy = 'omit')

Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77)

In [76]:
ttest_ind(democrat['budget'], republican['budget'], 
          nan_policy = 'omit')

Ttest_indResult(statistic=23.21277691701378, pvalue=2.0703402795404463e-77)

In [78]:
# pvalue is close to 1 meaning that both parties voted close to the same
ttest_ind(republican['water-project'], democrat['water-project'], 
          nan_policy = 'omit')

Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485)

In [79]:
# the pvalue is very small so one party voted stronger than the other
ttest_ind(republican['el-salvadaor-aid'], democrat['el-salvadaor-aid'], 
          nan_policy = 'omit')

Ttest_indResult(statistic=21.13669261173219, pvalue=5.600520111729011e-68)

In [80]:
# Shows that the republicans voted higher than the democrats
print(republican['el-salvadaor-aid'].mean())
print(democrat['el-salvadaor-aid'].mean())

0.9515151515151515
0.21568627450980393


In [89]:
# again the number is very small, rejecting the null hypothesis
ttest_ind(republican['duty-free'], democrat['duty-free'], 
          nan_policy = 'omit')

Ttest_indResult(statistic=-12.853146132542978, pvalue=5.997697174347365e-32)

In [82]:
# Shows that the democrats voted higher than the republicans
print(republican['duty-free'].mean())
print(democrat['duty-free'].mean())

0.08974358974358974
0.6374501992031872


In [120]:
# which parties voted higher for the bill?
for i, row in enumerate(republican.columns):
    m = ttest_ind(republican[row], democrat[row], 
          nan_policy = 'omit')
    repub_mean = republican[row].mean()
    demo_mean = democrat[row].mean()
    
    # if democrat party voted higher than the republican party, reject the null hypothesis
    if m[1] < .01 and demo_mean >repub_mean:
      print("Democratic Party voted higher than the Republican Party for", row)
      print("Average Republican votes:", republican[row].mean())
      print("Average Democratic votes:", democrat[row].mean())
      print(m, '\n')

    # if republican party voted higher than the democratic party, reject the null hypothesis
    elif m[1] < .01 and repub_mean > demo_mean:
      print('Republican Party voted higher than the Democratic Party for', row)
      print("Average Republican votes:", republican[row].mean())
      print("Average Democratic votes:", democrat[row].mean())
      print(m,'\n')

    # if both parties voted similarly, don't reject the null hypothesis
    else:
      print('Both Parties Voted Similarly for', row)
      print("Average Republican votes:", republican[row].mean())
      print("Average Democratic votes:", democrat[row].mean())
      print(m, '\n')

Democratic Party voted higher than the Republican Party for handicapped-infants
Average Republican votes: 0.18787878787878787
Average Democratic votes: 0.6046511627906976
Ttest_indResult(statistic=-9.205264294809222, pvalue=1.613440327937243e-18) 

Both Parties Voted Similarly for water-project
Average Republican votes: 0.5067567567567568
Average Democratic votes: 0.502092050209205
Ttest_indResult(statistic=0.08896538137868286, pvalue=0.9291556823993485) 

Democratic Party voted higher than the Republican Party for budget
Average Republican votes: 0.13414634146341464
Average Democratic votes: 0.8884615384615384
Ttest_indResult(statistic=-23.21277691701378, pvalue=2.0703402795404463e-77) 

Republican Party voted higher than the Democratic Party for physician-fee-freeze
Average Republican votes: 0.9878787878787879
Average Democratic votes: 0.05405405405405406
Ttest_indResult(statistic=49.36708157301406, pvalue=1.994262314074344e-177) 

Republican Party voted higher than the Democratic Pa